<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/improved_GAN/ACGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACGAN 

**ACGAN(보조 분류 GAN)** 은 원리상 Conditional GAN 과 유사한 형태를 가진다. CGAN 의 경우, 판별기는 입력으로 이미지와 그 레이블을 받고 그 이미지가 진짜일 확률을 출력한다. 그러나 ACGAN 에서의 입력은 이미지이고 출력은 이미지가 진짜인지의 확률과 어떤 클래스인지를 동시에 출력하게 된다. 

기본적으로 CGAN 에서는 부가정보를 네트워크에 제공하지만, ACGAN에서는 이를 제공하지 않고 보조 클래스 디코더 네트워크(Axuiliary class decoder)를 이용하여 부가 정보를 재구성한다.

ACGAN 에는 추가적으로 분류기의 손실함수가 주어지는데 진짜와 가짜 이미지를 추가적으로 분류하는 작업을 수행한다. 
또, 생성기의 손실함수에서도 판별기를 속이는 일 외에도 가짜이미지를 정확하게 분류했는지도 손실을 계산하게된다.

#GPU 할당

Colab 이 아닌 환경에서 아래의 코드를 통해 우선적으로 gpu를 할당해준다.

In [ ]:
import tensorflow as tf 
physical_devices =tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

import numpy as np
import math
import matplotlib.pyplot as plt

# 생성기 및 판별기 함수구성
CGAN 의 코드를 기반으로 하여 구성한다. 

In [1]:
def discriminator(inputs, activation='sigmoid', num_labels=None, num_codes=None):
  """
  판별기 모델구성 
  가짜와 진짜를 판별하기 위한 LeakyReLU - Conv2D 로 구성된 스택이다.
  이 네트워크는 BN 으로 수렴되지 않으므로, 사용하지 않는다.
  """

  # 입력 인수
  """
  inputs(layer) : 판별기의 입력 계층(이미지)
  activation (string) : 출력 활성화 계층의 이름
  num_labels (int): ACGAN 과 InfoGAN 에서의 원-핫 레이블의 차원
  num_codes(int) : Q network as output
  """

  #출력 인수 : Model: 판별기 모델

  kernel_size = 5
  layer_filters = [32, 64, 128, 256]

  x = inputs
  for filters in layer_filters:
    # 첫 세 개의 합성곱 계층에서는 strides = 2 를 사용
    # 마지막 한 계층은 strides = 1을 사용
    
    if filters == layer_filters[-1]:
      strides = 1
    else:
      strides = 2

    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(filters = filsters, kernel_size = kernel_size, strides=strides, padding='same')(x)

  x = Flatten()(x)
  outputs = Dense(1)(x)
  if activation is not None:
    outputs = Activation(activation)(outputs)

  if num_labels:
    # ACGAN 과 InfoGAN 에는 두번째 출력이 있음
    # MNIST 에서 두 번째 출력은 10차원 원-핫 레이블 벡터로 구성됨

    layer = Dense(layer_filters[-2])(x)
    labels = Dense(num_labels)(layer)
    labels - Activation('softmax',name='label')(labels)
    if num_code is None:
      outputs = [outputs, labels]

    else:
      #InfoGAN 에는 세 번쨰와 네 번째 출력이 있음
      #세 번째 출력: x 가 주어졌을 때 첫번째 c 를 나타내는 1차원 연속 Q

      code1 = Dense(1)(layer)
      code1 = Activation('sigmoid', name='code1')(code1)

      #네 번째 출력: x가 주어졌을 때 두 번째 c를 나타내는 1차원 연속 Q
      code2 = Dense(1)(layer)
      code2 = Activation('sigmoid',name='code2')(code2)

      outputs = [outputs, labels, code1, code2]

  elif num_codes is not None:
    #z0_recon: z0 정규 분포를 재구성
    z0_recon = Dense(num_codes)(x)
    z0_recon = Activation('tanh', name='z0')(z0_recon)

    outputs = [outputs, z0_recon]

  return Model(inputs, outputs, name='discriminator')

In [ ]:
def generator(inputs, image_size, activation='sigmoid', labels=None, codes=None):
  